In [25]:
import pandas as pd

# Baca hasil gabungan sebelumnya
df = pd.read_csv('data_cuaca_gabungan.csv')

print(df.columns.tolist())

['station_id', 'station_name', 'current_latitude', 'current_longitude', 'timestamp', 'wind_dir_deg_dd', 'wind_speed_ff', 'visibility_vv', 'present_weather_ww', 'past_weather_w1', 'past_weather_w2', 'pressure_qfe_mb_derived', 'pressure_qff_mb_derived', 'temp_drybulb_c_tttttt', 'temp_dewpoint_c_tdtdtd', 'temp_wetbulb_c', 'temp_max_c_txtxtx', 'temp_min_c_tntntn', 'relative_humidity_pc', 'rainfall_last_mm', 'rainfall_24h_rrrr', 'rainfall_6h_rrr', 'evaporation_24hours_mm_eee', 'sunshine_h_sss', 'qc_flag', 'qc_parameter', 'rainfall_indicator_ir', 'evaporation_eq_indicator_ie', 'station_id;station_name;current_latitude;current_longitude;timestamp;wind_dir_deg_dd;wind_speed_ff;visibility_vv;present_weather_ww;past_weather_w1;past_weather_w2;pressure_qfe_mb_derived;pressure_qff_mb_derived;temp_drybulb_c_tttttt;temp_dewpoint_c_tdtdtd;temp_wetbulb_c;temp_max_c_txtxtx;temp_min_c_tntntn;relative_humidity_pc;rainfall_last_mm;rainfall_24h_rrrr;rainfall_6h_rrr;evaporation_24hours_mm_eee;sunshine_h_sss

In [28]:
import pandas as pd

# 1. Baca file asli
df = pd.read_csv("data_cuaca_gabungan.csv")

# 2. Pilih kolom yang dibutuhkan
kolom_dipakai = [
    'timestamp',
    'temp_drybulb_c_tttttt',
    'relative_humidity_pc',
    'pressure_qff_mb_derived',
    'wind_speed_ff',
    'wind_dir_deg_dd',
    'rainfall_last_mm',
    'present_weather_ww'
]

df_singkat = df[kolom_dipakai].copy()

# 3. Pisahkan timestamp jadi Tanggal dan Jam
df_singkat['Tanggal'] = pd.to_datetime(df_singkat['timestamp'], errors='coerce').dt.date
df_singkat['Jam'] = pd.to_datetime(df_singkat['timestamp'], errors='coerce').dt.time
df_singkat.drop(columns=['timestamp'], inplace=True)

# 4. Ganti nama kolom agar sesuai tampilan web
df_singkat.rename(columns={
    'temp_drybulb_c_tttttt': 'Suhu (°C)',
    'relative_humidity_pc': 'Kelembapan (%)',
    'pressure_qff_mb_derived': 'Tekanan (hPa)',
    'wind_speed_ff': 'Kecepatan Angin',
    'wind_dir_deg_dd': 'Arah Angin',
    'rainfall_last_mm': 'Curah Hujan',
    'present_weather_ww': 'Kondisi Cuaca'
}, inplace=True)

# 🆕 4.1 Mapping kondisi cuaca dari kode ke deskripsi teks
kode_ww = {
    0: "Cerah",
    1: "Berawan",
    2: "Cerah",
    3: "Berawan",
    45: "Kabut",
    60: "Hujan Ringan",
    61: "Hujan Sedang",
    63: "Hujan Sedang",
    65: "Hujan",
    80: "Hujan",
    95: "Hujan Petir",
    99: "Hujan Deras Petir"
}

# Pastikan kolom 'Kondisi Cuaca' dalam bentuk angka (float), lalu mapping ke deskripsi
df_singkat['Kondisi Cuaca'] = df_singkat['Kondisi Cuaca'].astype(float).map(kode_ww)
df_singkat['Kondisi Cuaca'] = df_singkat['Kondisi Cuaca'].fillna("Cerah")

# 5. Hapus baris dengan data penting yang kosong
kolom_wajib = [
    'Suhu (°C)', 'Kelembapan (%)', 'Tekanan (hPa)',
    'Kecepatan Angin', 'Arah Angin',
    'Kondisi Cuaca', 'Tanggal', 'Jam'
]
df_singkat.dropna(subset=kolom_wajib, inplace=True)

# 6. Isi kosong pada 'Curah Hujan' dengan 0.0 dan bersihkan nilai aneh seperti 8888.0
df_singkat['Curah Hujan'] = df_singkat['Curah Hujan'].fillna(0.0)
df_singkat['Curah Hujan'] = df_singkat['Curah Hujan'].replace(8888.0, 0.0)

# 6.1 Hapus data duplikat, hanya simpan 1 baris dari setiap kombinasi
df_singkat.drop_duplicates(inplace=True)

# 7. Susun kolom sesuai urutan yang kamu minta
kolom_final = [
    'Tanggal', 'Jam',
    'Suhu (°C)', 'Kelembapan (%)', 'Tekanan (hPa)',
    'Kecepatan Angin', 'Arah Angin',
    'Curah Hujan', 'Kondisi Cuaca'
]
df_singkat = df_singkat[kolom_final]

# 8. Simpan ke file baru
df_singkat.to_csv("cuaca_web_final.csv", index=False)

print("✅ File 'cuaca_web_final.csv' berhasil dibuat dan dibersihkan.")

✅ File 'cuaca_web_final.csv' berhasil dibuat dan dibersihkan.


In [29]:
# Baca file CSV
df = pd.read_csv("cuaca_web_final.csv")

# Tampilkan jumlah nilai kosong di setiap kolom
print(df.isnull().sum())

# Jumlah baris
print("Jumlah baris:", len(df))

Tanggal            0
Jam                0
Suhu (°C)          0
Kelembapan (%)     0
Tekanan (hPa)      0
Kecepatan Angin    0
Arah Angin         0
Curah Hujan        0
Kondisi Cuaca      0
dtype: int64
Jumlah baris: 1926


In [30]:
import pandas as pd

# Baca file CSV final kamu
df = pd.read_csv("cuaca_web_final.csv")

# Buka file output
with open("data_values.sql", "w", encoding='utf-8') as f:
    for _, row in df.iterrows():
        nilai = (
            f"('{row['Tanggal']}', '{row['Jam']}', {row['Suhu (°C)']}, {row['Kelembapan (%)']}, "
            f"{row['Tekanan (hPa)']}, {row['Kecepatan Angin']}, {row['Arah Angin']}, "
            f"{row['Curah Hujan']}, '{row['Kondisi Cuaca']}'),\n"
        )
        f.write(nilai)

print("✅ File berisi isi data SQL saja berhasil dibuat.")

✅ File berisi isi data SQL saja berhasil dibuat.
